In [ ]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, log_loss, accuracy_score

import matplotlib.pyplot as plt
import seaborn as sns
import itertools

import tensorflow as tf
from tensorflow import keras
from keras import models
from keras import layers

from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical       # convert to one-hot-encoding
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
from keras.optimizers import RMSprop
from keras.callbacks import ReduceLROnPlateau

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train = pd.read_csv("../input/sign-language-mnist/sign_mnist_train/sign_mnist_train.csv")
test = pd.read_csv("../input/sign-language-mnist/sign_mnist_test/sign_mnist_test.csv")

In [ ]:
train.head()

In [ ]:
print(train.columns)
print(test.columns)

In [ ]:
print(train.shape)
print(test.shape)

In [ ]:
X = train.drop('label', axis=1)
y = train['label']

In [ ]:
X=np.array(X)
y=np.array(y)

In [ ]:
plt.imshow(X[3].reshape(28,28))
plt.show()

In [ ]:
X = X/255.0
x_train,x_test,y_train,y_test = train_test_split(X,y,test_size = 0.2)

In [ ]:
print(x_train.shape)
print(x_test.shape)

In [ ]:
re_x_train = x_train.reshape(-1,28,28,1)
re_x_test = x_test.reshape(-1,28,28,1)

In [ ]:
model = tf.keras.Sequential([

    Dropout(rate= 0.01),
    Conv2D(filters = 64, kernel_size = (5,5),padding = 'Same', activation ='relu', input_shape = (28,28,1)),
    MaxPool2D(pool_size=(2,2)),
    Dropout(0.25),
    
    Dense(256,activation = 'relu'),
    Conv2D(filters = 128, kernel_size = (5,5),padding = 'Same', activation ='relu'),
    MaxPool2D(pool_size=(2,2)),
    Dropout(0.25),
    
    Dense(512,activation = 'relu'),
    Conv2D(filters = 256, kernel_size = (5,5),padding = 'Same', activation ='relu'),
    MaxPool2D(pool_size=(2,2), strides=(2,2)),
    layers.Dropout(0.25),
    
    Flatten(),
    Dense(1024,activation = 'relu'), 
    BatchNormalization(),
    Dense(25,activation = 'softmax')
])


In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
              loss="sparse_categorical_crossentropy",
              metrics=['accuracy'])

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    min_delta=0, 
    patience=30, 
    verbose=0,
    mode='auto', 
    baseline=None, 
    restore_best_weights=False
)

In [ ]:
history = model.fit(
    re_x_train,
    y_train,
    callbacks = early_stopping,
    epochs=30,
    validation_data = (re_x_test,y_test)
)

In [ ]:
history_df = pd.DataFrame(history.history)
history_df.loc[:, ['accuracy','val_accuracy']].plot()
history_df.loc[:, ['loss','val_loss']].plot()

In [ ]:
ANS = test['label']
test1 = test.drop('label', axis=1)
test2 = test1/255.0

In [ ]:
re_test = test2.values.reshape(-1,28,28,1)
pred = model.predict(re_test)

PRED=[]
for item in pred:
    value2=np.argmax(item)      
    PRED+=[value2]

In [ ]:
accuracy = accuracy_score(ANS,PRED)
print(accuracy)